In [ ]:
import pandas as pd

In [ ]:
import json
from dateutil.parser import parse

# Load the data
df = pd.read_csv(r'C:\Users\mcaballero\Desktop\auto_datagencars\resources\dataset_imascono\data_imascono.csv')

# Transform user_id from string to numeric values
user_ids = df['user_id'].unique()
user_id_dict = {user_id: i+1 for i, user_id in enumerate(user_ids)}
df['user_id'] = df['user_id'].map(user_id_dict)

# user.csv
df_user = df[['user_id']].drop_duplicates()
df_user.to_csv(r'C:\Users\mcaballero\Desktop\auto_datagencars\resources\dataset_imascono\user.csv', index=False)

# item.csv
df_item = df[df['object_type'] != 'Player'][['object_id', 'object_type', 'object_action', 'object_position', 'room_id']].drop_duplicates()
df_item.columns = ['item_id', 'object_type', 'object_action_types', 'object_position', 'room_id']

grouped_data = df_item.groupby('object_type')['object_action_types'].agg(lambda x: x.unique())
for object_type, object_actions in grouped_data.items():
    df_item.loc[df_item['object_type'] == object_type, 'object_action_types'] = ', '.join(object_actions)

# Remove all rows where item_id is not an integer
df_item = df_item[pd.to_numeric(df_item['item_id'], errors='coerce').notna()]

# Keep the row with object_position when dropping item_id duplicates
df_item = df_item.sort_values('object_position', ascending=False).drop_duplicates(subset='item_id', keep='first')

# Remove rows where item_id is "position", object_action_types is "player", and object_position is "update"
df_item = df_item[~((df_item['item_id'] == 'position') & 
                    (df_item['object_action_types'] == 'player') & 
                    (df_item['object_position'] == 'update'))]

# Sort the dataframe by item_id
df_item = df_item.sort_values('item_id')

df_item.to_csv(r'C:\Users\mcaballero\Desktop\auto_datagencars\resources\dataset_imascono\item.csv', index=False)

# context.csv
df['context_id'] = df.groupby('device_data').ngroup() + 1
context_csv = df[['context_id', 'device_data']].drop_duplicates()

# Transforming the JSON strings in device_data into dictionary
context_csv['device_data'] = context_csv['device_data'].apply(lambda x: json.loads(x) if pd.notnull(x) else "{}")

context_csv.to_csv(r'C:\Users\mcaballero\Desktop\auto_datagencars\resources\dataset_imascono\context.csv', index=False)

# behavior.csv
behavior_csv = df[['user_id', 'object_action', 'object_position', 'object_id', 'context_id', '@timestamp']]
behavior_csv.columns = ['user_id', 'object_action', 'user_position', 'item_id', 'context_id', 'timestamp']

# Convert timestamp column to datetime object and format it
behavior_csv['timestamp'] = behavior_csv['timestamp'].apply(lambda x: parse(x).strftime('%Y-%m-%d %H:%M:%S'))

behavior_csv['behavior_id'] = behavior_csv.index + 1
behavior_csv.to_csv(r'C:\Users\mcaballero\Desktop\auto_datagencars\resources\dataset_imascono\behavior.csv', index=False)

In [ ]:
# PROCESS BEHAVIOR.CSV

df = pd.read_csv(r'C:\Users\mcaballero\Desktop\auto_datagencars\resources\dataset_imascono\behavior.csv')

# Remove rows where 'context_id' is None
df = df[df['context_id'].notna()]

df.to_csv(r'C:\Users\mcaballero\Desktop\auto_datagencars\resources\dataset_imascono\behavior2.csv', index=False)